## Kaggle data management api

In [1]:
# !pip install kaggle # now download the api token and store it to /home/ec2-user/.kaggle/kaggle.json
# !cd /home/ec2-user/SageMaker
# !mkdir /home/ec2-user/.kaggle/
# !mv /home/ec2-user/SageMaker/kaggle.json /home/ec2-user/.kaggle/kaggle.json
# !chmod 600 /home/ec2-user/.kaggle/kaggle.json # for privacy
!df -h --total  # get the harddisk info
!nvcc --version  # Get the cuda version


Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         30G   72K   30G   1% /dev
tmpfs            30G     0   30G   0% /dev/shm
/dev/xvda1       94G   78G   17G  83% /
/dev/xvdf       985G  535G  411G  57% /home/ec2-user/SageMaker
total           1.2T  612G  487G  56% -
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [2]:
# !git clone https://github.com/RamsteinWR/PneumoniaRSNA1.git
# !rm -rf /home/ec2-user/SageMaker/PneumoniaRSNA1

In [3]:
%%time
import os
os.chdir("/home/ec2-user/SageMaker")

# Download the challenge data here 
# !kaggle competitions download -c rsna-pneumonia-detection-challenge
# !unzip -o -q /home/ec2-user/SageMaker/rsna-pneumonia-detection-challenge.zip -d /home/ec2-user/SageMaker/PneumoniaData  #-q for quitely no verbose
# !rm -rf /home/ec2-user/SageMaker/rsna-pneumonia-detection-challenge.zip

CPU times: user 49 µs, sys: 0 ns, total: 49 µs
Wall time: 54.1 µs


In [5]:
# !mv /home/ec2-user/SageMaker/PneumoniaData/stage_2_detailed_class_info.csv /home/ec2-user/SageMaker/PneumoniaData/detailed_class_info.csv
# !mv /home/ec2-user/SageMaker/PneumoniaData/stage_2_train_labels.csv /home/ec2-user/SageMaker/PneumoniaData/train_labels.csv
# !mv /home/ec2-user/SageMaker/PneumoniaData/stage_2_train_images /home/ec2-user/SageMaker/PneumoniaData/train_dicoms
# !mv /home/ec2-user/SageMaker/PneumoniaData/stage_2_test_images /home/ec2-user/SageMaker/PneumoniaData/test_dicoms

## Preprocessing the authenication management

In [4]:
%%time
import boto3
import botocore
from botocore.exceptions import ClientError
from tqdm import tqdm
import os 
import urllib.request
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = "brain-imaging-intracranial-hemorrhage"
region_name="us-east-1"
aws_access_key_id = "hfaisdufhiu"
aws_secret_access_key = "fudshfuh"

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)    
    data = open(file, "rb")
    key = channel + '/' + file
    print("Uploading file {} to s3://{}/{}".format(file, bucket, channel))
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def upload_dir_to_s3(bucket, s3_folder, dir_to_upload):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
    print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_folder))
    # enumerate local files recursively
    for root, dirs, files in os.walk(dir_to_upload):
        for filename in tqdm(files):
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, dir_to_upload)
            s3_path = os.path.join(s3_folder, relative_path).replace("\\", "/")
            try:
                s3_client.head_object(Bucket=bucket, Key=s3_path)
                print("Path found on S3! Deleting %s..." % s3_path)
                try:
                    s3_client.delete_object(Bucket=bucket, Key=s3_path)
                    try:
#                         print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_path)
                        s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                    except ClientError as e:
                        logging.error(e)
                except:
                    print("Unable to delete from s3 %s..." % s3_path)
            except:
                try:
                    s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                except ClientError as e:
                    logging.error(e)
    print("Upload completed successfully.")
    
def download_dir(s3_folder, local_path, bucket=""):
    """
    params:
    - s3_folder: pattern to match in s3
    - local_path: local_path path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    client = boto3.client('s3', region_name=region_name)
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket': bucket,
        'Prefix': s3_folder,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local_path, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    print("{} files found in {} directories. Downloading now...".format(len(keys), len(dirs)))
    for k in tqdm(keys):
        dest_pathname = os.path.join(local_path, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        try:
#             print("Downloading {}".format(dest_pathname))
            client.download_file(bucket, k, dest_pathname)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
    print("{} files downloaded successfully.".format(len(keys)))
    
# download('http://data.lip6.fr/cadene/pretrained/home/ec2-user/SageMaker/rsna19/models/se_resnext50_32x4d-a260b3a4.pth')
# upload_to_s3("pretrained", 'se_resnext50_32x4d-a260b3a4.pth')
# upload_dir_to_s3(bucket=bucket, s3_folder='aptos-2019', dir_to_upload="/home/ec2-user/SageMaker/data")

# download_dir(s3_folder='aptos-2015/train_images_768/', local_path='/home/ec2-user/SageMaker/data/aptos-2015/train_images_768/', bucket=bucket)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


CPU times: user 1.07 s, sys: 517 ms, total: 1.59 s
Wall time: 1.87 s


## Uploading files to s3

<a id='idg4c.1'></a>
## Dependencies
___
### import packages and check SageMaker version

In [5]:
!pip install -r /home/ec2-user/SageMaker/PneumoniaRSNA1/requirements.txt
# !pip install tensorflow --upgrade --force-reinstall

  Using cached easydict-1.9-py3-none-any.whl
  Using cached Keras-2.2.0-py2.py3-none-any.whl (300 kB)
  Using cached mxnet_cu100-1.7.0-py2.py3-none-manylinux2014_x86_64.whl (827.8 MB)
  Using cached networkx-2.5-py3-none-any.whl (1.6 MB)
  Using cached opencv_python-3.4.1.15-cp36-cp36m-manylinux1_x86_64.whl (24.9 MB)
  Using cached pandas-0.23.4-cp36-cp36m-manylinux1_x86_64.whl (8.9 MB)
  Using cached Pillow-5.0.0-cp36-cp36m-manylinux1_x86_64.whl (5.9 MB)
  Using cached pydicom-1.2.0-py2.py3-none-any.whl (7.3 MB)
  Using cached pytest-6.2.1-py3-none-any.whl (279 kB)
  Using cached scikit_image-0.14.1-cp36-cp36m-manylinux1_x86_64.whl (25.3 MB)
  Using cached scikit_learn-0.20.0-cp36-cp36m-manylinux1_x86_64.whl (5.3 MB)
  Using cached scipy-1.1.0-cp36-cp36m-manylinux1_x86_64.whl (31.2 MB)
  Using cached tensorflow_gpu-1.4.0-cp36-cp36m-manylinux1_x86_64.whl (170.3 MB)
  Using cached toposort-1.6-py2.py3-none-any.whl (9.5 kB)
  Using cached enum34-1.1.10-py3-none-any.whl (11 kB)
  Using ca

In [6]:
import json
# import torch
import tarfile
import pickle
import matplotlib.pyplot as plt
# import torchvision as tv
import pathlib                          # Path management tool (standard library)
import subprocess                       # Runs shell commands via Python (standard library)
import sagemaker                        # SageMaker Python SDK
# from sagemaker.pytorch import PyTorch   # PyTorch Estimator for TensorFlow

## Data processing

In [8]:
# import os, sys
# os.chdir("/home/ec2-user/SageMaker/PneumoniaRSNA1")
# # sys.path.append("/home/ec2-user/SageMaker/HaemorrhageRSNA19")

# !sh /home/ec2-user/SageMaker/PneumoniaRSNA1/prepare_data.sh

# os.chdir("/home/ec2-user/SageMaker")

### Downloading trained models, contains codes, model weights, and pretrained models

In [12]:
# !wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=12abFXy7-FOwoKxFSJ__IbOGm9oQDU7CQ' -O /home/ec2-user/SageMaker/PneumoniaRSNA1/models.tar.gz

### if not models.tar.gz: download pretarined models here

In [11]:
# !wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1rI_WSlot6ZNa_ERdLSCsGquUXEK_ikYb' -O /home/ec2-user/SageMaker/PneumoniaRSNA1/pretrained.zip

In [18]:
# !tar -xvzf /home/ec2-user/SageMaker/PneumoniaRSNA1/models.tar.gz -C /home/ec2-user/SageMaker/PneumoniaRSNA1/model_ckpt

In [19]:
# !unzip -o -q /home/ec2-user/SageMaker/PneumoniaRSNA1/pretrained.zip -d /home/ec2-user/SageMaker/PneumoniaRSNA1/models/pretrained  #-q for quitely no verbose

## Training the simple model
### adding some modules to path

In [7]:
import sys
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/models")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/models/RelationNetworks")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/models/DeformableConvNets")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/models/RetinaNet")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/src/etl")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/src/GrayscaleModules")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/src/infer")
sys.path.append("/home/ec2-user/SageMaker/PneumoniaRSNA1/src/train")
print(sys.path)

['', '/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python36.zip', '/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6', '/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/lib-dynload', '/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages', '/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ec2-user/.ipython', '/home/ec2-user/SageMaker/PneumoniaRSNA1/models', '/home/ec2-user/SageMaker/PneumoniaRSNA1/models/RelationNetworks', '/home/ec2-user/SageMaker/PneumoniaRSNA1/models/DeformableConvNets', '/home/ec2-user/SageMaker/PneumoniaRSNA1/models/RetinaNet', '/home/ec2-user/SageMaker/PneumoniaRSNA1/src/etl', '/home/ec2-user/SageMaker/PneumoniaRSNA1/src/GrayscaleModules', '/home/ec2-user/SageMaker/PneumoniaRSNA1/src/infer', '/home/ec2-user/SageMaker/PneumoniaRSNA1/src/train']


In [8]:
# !pip uninstall tensorflow protobuf --yes
# !find $CONDA_PREFIX -name "tensorflow" | xargs -Ipkg rm -rfv pkg
# !pip install --ignore-installed --upgrade https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-1.9.0-cp36-cp36m-linux_x86_64.whl --no-cache-dir


In [9]:
import os
os.chdir("/home/ec2-user/SageMaker/PneumoniaRSNA1")

# !sh /home/ec2-user/SageMaker/PneumoniaRSNA1/train_simple.sh
!python src/train/TrainOneClassifier.py

os.chdir("/home/ec2-user/SageMaker")

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.8.

## Generating predictions for challenge data 1/2 (Dmytro's models)
### Run the following set of commands for folds 0-4:

In [ ]:
# Stage 1 models out-of-fold predictions
!python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnet18_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnet34_400_5_planes_combine_last_var_dr0 --epoch 7 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf3D/predict.py predict_oof --model dpn68_384_5_planes_combine_last --epoch 72 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnet18_384_5_planes_bn_f8 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model airnet50_384 --epoch 6 --fold 0 --mode all

# Stage 1 models test predictions
!python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnet18_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnet34_400_5_planes_combine_last_var_dr0 --epoch 7 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf3D/predict.py predict_test --model dpn68_384_5_planes_combine_last --epoch 72 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnet18_384_5_planes_bn_f8 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model airnet50_384 --epoch 6 --fold 0 --mode all

# # Stage 2 models out-of-fold predictions
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model se_preresnext26b_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnext50_400 --epoch 6 --fold 0 --mode all

# # Stage 2 models test predictions
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model se_preresnext26b_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnext50_400 --epoch 6 --fold 0 --mode all